# mcPECASO#

### File to generate Fig 3, Fig S3###

# Objective: Productivity #

### Import necessary modules and adjust settings ###

In [2]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as pecaso
pecaso.settings.num_points = 25
pecaso.settings.parallel=True
pecaso.settings.time_end = 100
pecaso.settings.initial_biomass = 0.05
pecaso.settings.initial_substrate = 500
pecaso.settings.num_timepoints = 10000
pecaso.settings.scope = 'global'

num_cores = multiprocessing.cpu_count()
model = cameo.models.bigg.iJO1366


Academic license - for non-commercial use only


In [3]:
metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol<br>3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxy<br>propanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxy-<br>acetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthio-<br>ribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}
data = {'names':[metabolite_names_dict[product] for product in carbon_dict.keys()], 'number':list(carbon_dict.values())}
metabolite_df = pd.DataFrame(data,index=list(carbon_dict.keys()))
metabolite_order = metabolite_df.sort_values(by=['number','names']).names.values

In [4]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


# Uptake: Logistic#

In [5]:
model = cameo.models.bigg.iJO1366

pecaso.settings.uptake_fun = 'logistic'
pecaso.settings.objective = 'batch_productivity'

pecaso.settings.uptake_params = {'B': 0}
pecaso_constant_global = mcPECASO(model=model,biomass_rxn=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                           substrate_rxn=model.reactions.EX_glc__D_e, target_rxn=model.reactions.EX_lac__D_e,
                           condition='Constant Substrate Uptake')

pecaso.settings.uptake_params = {'B': 5}
pecaso_reduced_global = mcPECASO(model=model,biomass_rxn=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                          substrate_rxn=model.reactions.EX_glc__D_e, target_rxn=model.reactions.EX_lac__D_e,
                          condition='Reduced Substrate Uptake')

pecaso_constant_extrema = deepcopy(pecaso_constant_global)
pecaso_constant_extrema.settings.scope = 'extrema'

pecaso_reduced_extrema = deepcopy(pecaso_reduced_global)
pecaso_reduced_extrema.settings.scope = 'extrema'

pecaso.settings.uptake_fun = 'linear'
pecaso.settings.uptake_params = {'m': 12.468033225283747,
                                   'c': 0.796100176673716}
pecaso_linear_global = mcPECASO(model=model,biomass_rxn=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                                substrate_rxn=model.reactions.EX_glc__D_e, target_rxn=model.reactions.EX_lac__D_e,
                                condition='Reduced Substrate Uptake')

pecaso_linear_extrema = deepcopy(pecaso_linear_global)
pecaso_linear_extrema.settings.scope = 'extrema'

The model is complete.
The model is complete.


c:\users\kraj5\onedrive - university of toronto\research\phd_uoft\python\tsdyssco\tsdyssco\core\substrate_dependent_envelopes.py:72: UserWarning:

The parameters used with the model for substrate uptake resulted in rates that are lower than thee minimum feasible uptake for one or more cases. The minimum feasible uptake rate was used in these cases



The model is complete.


In [6]:
pecaso_constant_global.calculate_fermentation_characteristics()
pecaso_reduced_global.calculate_fermentation_characteristics()
pecaso_linear_global.calculate_fermentation_characteristics()
pecaso_constant_extrema.calculate_fermentation_characteristics()
pecaso_reduced_extrema.calculate_fermentation_characteristics()
pecaso_linear_extrema.calculate_fermentation_characteristics()


Starting parallel pool


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    5.8s
[Parallel(n_jobs=8)]: Done  16 out of  25 | elapsed:    5.9s remaining:    3.3s
[Parallel(n_jobs=8)]: Done  22 out of  25 | elapsed:    5.9s remaining:    0.7s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    5.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 256 tasks      | elapsed:    4.0s
[Parallel(n_jobs=8)]: Done 625 out of 625 | elapsed:   10.5s finished


Completed analysis in  16.62044906616211 s
Starting parallel pool


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   4 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  16 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 366 tasks      | elapsed:    5.5s
[Parallel(n_jobs=8)]: Done 610 out of 625 | elapsed:    9.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done 625 out of 625 | elapsed:    9.7s finished


Completed analysis in  9.79411244392395 s
Starting parallel pool


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   4 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  16 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 361 tasks      | elapsed:    5.3s
[Parallel(n_jobs=8)]: Done 610 out of 625 | elapsed:    8.7s remaining:    0.1s
[Parallel(n_jobs=8)]: Done 625 out of 625 | elapsed:    9.3s finished


Completed analysis in  9.427870273590088 s
Completed analysis in  27.21509838104248 s
Completed analysis in  25.732053518295288 s
Completed analysis in  26.501535654067993 s


# Production Envelopes #

In [8]:
with model:
    model.objective = 'EX_glc__D_e'
    min_uptake = -model.optimize().objective_value
    display(min_uptake)

0.13404255319148994

In [9]:
pecaso_list = [pecaso_constant_global, pecaso_reduced_global]

num_of_conditions = len(pecaso_list)
condition_list = [pecaso.condition for pecaso in pecaso_list]
envelope_dict = {condition: pecaso.production_envelope
                 for condition, pecaso in zip(condition_list, pecaso_list)}
colors = ['#397FB9','#E21E26']
max_growth = max([max(envelope['growth_rates']) for envelope in list(envelope_dict.values())])
max_uptake = max([max(envelope['substrate_uptake_rates']) for envelope in list(envelope_dict.values())])
max_flux = max([max(envelope['production_rates_ub']) for envelope in list(envelope_dict.values())])
max_yield = max([max(envelope['yield_ub']) for envelope in list(envelope_dict.values())])

fig = subplots.make_subplots(rows=1, cols=3, subplot_titles=
                              ['Substrate Uptake Rate', 'Product Flux', 'Product Yield'], horizontal_spacing=0.1)

line = [{'color': colors[0], 'width':2}, {'color': colors[1], 'dash': 'dashdot', 'width':2}]

fig.add_trace(go.Scatter(x=np.linspace(0,1,20),
                         y=np.linspace(min_uptake,10,20),
                             line={'color': 'black',
                                   'width': 2, 'dash': 'dash'}, name='Minimum Substrate Uptake Rate',
                             mode='lines', showlegend=False), 1, 1)

for i, condition in enumerate(envelope_dict):
    
    fig.add_trace(go.Scatter(x=list(envelope_dict[condition]['growth_rates']),
                             y=list(envelope_dict[condition]['substrate_uptake_rates']),
                             line=line[i], name=condition,
                             mode='lines', legendgroup=condition), 1, 1)
    fig.add_trace(go.Scatter(x=list((envelope_dict[condition]['growth_rates'])),
                             y=list(envelope_dict[condition]['production_rates_ub']),
                             line=line[i],
                             mode='lines', showlegend=False, legendgroup=condition, name=condition), 1, 2)
    fig.add_trace(go.Scatter(x=list((list(envelope_dict[condition]['growth_rates']))),
                             y=list(envelope_dict[condition]['yield_ub']),
                             line=line[i],
                             mode='lines', showlegend=False, legendgroup=condition, name=condition), 1, 3)

fig.update_yaxes(title_text='Substrate Uptake Rate<br>(mmol/gdw.h)', range=[0,11], row=1, col=1)
fig.update_yaxes(title_text='Product Flux<br>(mmol/gdw.h)', range=[0,22], row=1, col=2)
fig.update_yaxes(title_text='Product Yield<br>(mmol/mmol substrate)', range=[0,2.2], row=1, col=3)


fig.update_xaxes(range=[0,1],
                 ticks='outside',
                 title = 'Growth Rate (1/h)',
                 dtick = 0.2,
                 showgrid=False,
                 tickcolor='black',
                 tickwidth=1.5,
                 tickfont=dict(color='black', size=14),
                 titlefont=dict(color='black', size=14),
                 title_standoff=0,
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)

fig.update_yaxes(ticks='outside',
                 showgrid=False,
                 tickcolor='black',
                 tickwidth=1.5,
                 tickfont=dict(color='black', size=14),
                 titlefont=dict(color='black', size=14),
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)


fig.update_annotations(dict(font=dict(size=18, color='black')))
fig.update_layout(width=925, height=400, 
                  legend=dict(x=-0.07, y=-0.2, orientation='h', font=dict(size=14, color='black')))
plot(fig)

#if not os.path.exists('images'):
#    os.mkdir('images')
#pio.write_image(fig, 'images/production_envelopes.svg')   

# Constant Uptake #

# Fermentation Characteristics#

In [12]:
pecaso = pecaso_constant_global
pecaso_extrema = pecaso_constant_extrema

ts_fermentations = pecaso.two_stage_fermentation_list

target_metabolite = list(pecaso.target_rxn.metabolites.keys())[0].name
formula_weight = list(pecaso.target_rxn.metabolites.keys())[0].formula_weight

attribute_names = ['batch_productivity', 'batch_yield', 'batch_titer']
characteristics = ['productivity', 'yield', 'titer']
units = ['(mmol/L.h)', '(mmol product/mmol substrate)', '(mmol/L)']

for row, characteristic in enumerate(characteristics):
    tracelist = list()
    tracelist.append(go.Scatter(x=np.linspace(0, max(pecaso.two_stage_characteristics
                                                     ['stage_one_growth_rate']), 10),
                                y=np.linspace(0, max(pecaso.two_stage_characteristics
                                                     ['stage_two_growth_rate']), 10), mode='lines',
                                name='One Stage Points',
                                line={'color': '#E5F5FC', 'dash': 'dash'},
                                showlegend=False))
    
    tracelist.append(go.Contour(z=pecaso.two_stage_characteristics[characteristic],
                                x=pecaso.two_stage_characteristics['stage_one_growth_rate'],
                                y=pecaso.two_stage_characteristics['stage_two_growth_rate'],
                                showlegend=False,
                                ncontours=20,
                                line_width=0.21,
                                contours=dict(coloring='heatmap', showlabels=True,
                                              labelfont=dict(size=11.5, color='white')),
                                colorbar=dict(title=characteristic.title()+' '+ units[row],
                                              titleside='right',
                                              titlefont=dict(size=14, color='black'),
                                              nticks=10,
                                              ticks='outside',
                                              tickwidth=1,
                                              tickfont=dict(size=14, color='black'),
                                              thickness=13,
                                              showticklabels=True,
                                              thicknessmode='pixels',
                                              len=1.04,
                                              lenmode='fraction',
                                              outlinewidth=1,
                                              outlinecolor='black')))
    tracelist.append(go.Scatter(x=[pecaso.two_stage_best_batch.stage_one_fluxes[0]],
                                y=[pecaso.two_stage_best_batch.stage_two_fluxes[0]],
                                mode='markers', hoverinfo='text', showlegend=True,
                                name='Best Two Stage Process',
                                text=['Best Two Stage Process<br>' + characteristic.title() + ': ' +
                                      str(round(getattr(pecaso.two_stage_best_batch,
                                                        attribute_names[row]), 3))],
                                marker=dict(color='#AAC644',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))
    
    
    tracelist.append(go.Scatter(x=[pecaso.one_stage_best_batch.fluxes[0]],
                                y=[pecaso.one_stage_best_batch.fluxes[0]],
                                name='Best One Stage Process',
                                mode='markers', hoverinfo='text', showlegend=True,
                                text=['Best One Stage Process<br>' + characteristic.title() + ': ' +
                                      str(round(getattr(pecaso.one_stage_best_batch,
                                                        attribute_names[row]), 3))],
                                marker=dict(color='#9DD7F5',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))
    
    tracelist.append(go.Scatter(x=[max(pecaso_extrema.production_envelope.growth_rates)],
                                y=[min(pecaso_extrema.production_envelope.growth_rates)],
                                name='Traditional Two Stage Process',
                                mode='markers', hoverinfo='text', showlegend=True,
                                text=['Traditional Two Stage Process<br>' +
                                      characteristic.title() + ': ' +
                                      str(round(getattr(pecaso.two_stage_suboptimal_batch,
                                                        attribute_names[row]), 3))],
                                marker=dict(color='#F16263',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))

    fig = go.Figure(data=tracelist)
    
    fig.update_layout(hovermode='closest',
                      height=525,
                      width=465,
                      showlegend=True,
                      legend=dict(x=-0.2, y=-0.15, orientation='h',
                                  font=dict(size=14, color='black')))
    fig.update_xaxes(range=[0,1], 
                     dtick=0.2,
                     title_text='Stage 1 Growth Rate (1/h)',
                     title_font=dict(size=14, color='black'),
                     tickfont=dict(size=14, color='black'),
                     ticks='outside',
                     title_standoff=0,
                     tickwidth=1.5,
                     showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)
    
    fig.update_yaxes(range=[0,1], 
                     dtick=0.2,
                     title_text='Stage 1 Growth Rate (1/h)',
                     title_font=dict(size=14, color='black'),
                     tickfont=dict(size=14, color='black'),
                     ticks='outside',
                     tickwidth=1.5,
                     showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)

    plot(fig)
    
    #if not os.path.exists('images'):
    #    os.mkdir('images')
    #pio.write_image(fig, 'images/heatmap_'+characteristic+'_objA_const_uptake.svg')   

# Fermentation Trajectory#

In [13]:
pecaso = pecaso_constant_global
ts_suboptimal = pecaso.two_stage_suboptimal_batch
os_best = pecaso.one_stage_best_batch
ts_best = pecaso.two_stage_best_batch
ferm_list = [ts_suboptimal, os_best, ts_best]
titles = ['Traditional Two Stage<br>Process', 'Best One Stage<br>Process', 'Best Two Stage<br>Process']
fig = subplots.make_subplots(rows=1, cols=3, subplot_titles=titles, horizontal_spacing=0.1)
max_conc = max([(max([max(data) for data in ferm.data])) for ferm in ferm_list])
max_t = max([max(ferm.time) for ferm in ferm_list])
fig.update_annotations(dict(font=dict(size=18, color='black')))

for col, ferm in enumerate(ferm_list):
    if ferm:
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[0], name='Biomass Concentration<br>(g/L)',
                                    line={'color': '#CD7929', 'dash':'dash'}, legendgroup='Biomass',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col+1)
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[1], name='Substrate Concentration<br>(mmol/L)',
                                    line={'color': '#2278B5'}, legendgroup='Substrate',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col + 1)
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[2], name='Product Concentration<br>(mmol/L)',
                                    line={'color': '#5E9A42', 'dash':'dashdot'}, legendgroup='Product',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col + 1)
        if col != 1:
            fig.append_trace(go.Scatter(x=[ferm.optimal_switch_time]*30,
                                        y=np.linspace(0, 0.8*max_conc, 30),
                                        name='Optimal Switch Time', mode='lines',
                                        line={'color': 'black',
                                                'width': 2,
                                                'dash': 'dot'},
                                        showlegend=True if col == 2 else False), 1, col+1)
        fig['layout']['annotations'] = list(fig['layout']['annotations']) + \
                                       [go.layout.Annotation
                                        (dict(x=0.7*max_t,
                                              y=1.01*max_conc,
                                              xref='x'+str(col+1), yref='y'+str(col+1),
                                              text='Productivity=' + str(round(ferm.batch_productivity, 3)) +
                                              '<br>Yield=' + str(round(ferm.batch_yield, 3)) +
                                              '<br>End Titer=' + str(round(ferm.batch_titer,3)),
                                              showarrow=False, font=dict(size=12, color='black'),
                                              ax=-0, ay=-0))]



fig.update_layout(hovermode='closest',
                  height=475,
                  width=925,
                  showlegend=True,
                  legend=dict(x=-0.08, y=-0.15, orientation='h',
                              font=dict(size=14, color='black')))

fig.update_xaxes(range=[0,max_t], 
                 dtick=5,
                 title_font=dict(size=14, color='black'),
                 title_text='Time (h)',
                 tickfont=dict(size=14, color='black'),
                 ticks='outside',
                 tickwidth=1.5,
                 title_standoff=0,
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False,
                 showgrid=False)

fig.update_yaxes(range=[0,max_conc*1.08], 
                 title_text='Concentration<br>(mmol/L or g/L)',
                 title_font=dict(size=14, color='black'),
                 tickfont=dict(size=14, color='black'),
                 ticks='outside',
                 tickwidth=1.5,
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False,
                 showgrid=False)


plot(fig)

#if not os.path.exists('images'):
#    os.mkdir('images')
#pio.write_image(fig, 'images/fermentation_characteristics_objA_const_uptake.svg')   

# Logistic Uptake #

# Fermentation Characteristics#

In [15]:
pecaso = pecaso_reduced_global
pecaso_extrema = pecaso_reduced_extrema

ts_fermentations = pecaso.two_stage_fermentation_list

target_metabolite = list(pecaso.target_rxn.metabolites.keys())[0].name
formula_weight = list(pecaso.target_rxn.metabolites.keys())[0].formula_weight

attribute_names = ['batch_productivity', 'batch_yield', 'batch_titer']
characteristics = ['productivity', 'yield', 'titer']
units = ['(mmol/L.h)', '(mmol product/mmol substrate)', '(mmol/L)']

for row, characteristic in enumerate(characteristics):
    tracelist = list()
    tracelist.append(go.Scatter(x=np.linspace(0, max(pecaso.two_stage_characteristics
                                                     ['stage_one_growth_rate']), 10),
                                y=np.linspace(0, max(pecaso.two_stage_characteristics
                                                     ['stage_two_growth_rate']), 10), mode='lines',
                                name='One Stage Points',
                                line={'color': '#E5F5FC', 'dash': 'dash'},
                                showlegend=False))
    
    tracelist.append(go.Contour(z=pecaso.two_stage_characteristics[characteristic],
                                x=pecaso.two_stage_characteristics['stage_one_growth_rate'],
                                y=pecaso.two_stage_characteristics['stage_two_growth_rate'],
                                showlegend=False,
                                ncontours=20,
                                line_width=0.21,
                                contours=dict(coloring='heatmap', showlabels=True,
                                              labelfont=dict(size=11.5, color='white')),
                                colorbar=dict(title=characteristic.title()+' '+units[row],
                                              titleside='right',
                                              titlefont=dict(size=14, color='black'),
                                              nticks=10,
                                              ticks='outside',
                                              tickwidth=1,
                                              tickfont=dict(size=14, color='black'),
                                              thickness=13,
                                              showticklabels=True,
                                              thicknessmode='pixels',
                                              len=1.04,
                                              lenmode='fraction',
                                              outlinewidth=1,
                                              outlinecolor='black')))
    tracelist.append(go.Scatter(x=[pecaso_extrema.two_stage_best_batch.stage_one_factor/100*\
                                   max(pecaso_extrema.production_envelope.growth_rates)],
                                y=[pecaso_extrema.two_stage_best_batch.stage_two_factor/100*\
                                   max(pecaso_extrema.production_envelope.growth_rates)],
                                mode='markers', hoverinfo='text', showlegend=True,
                                name='Best Two Stage Process',
                                marker=dict(color='#AAC644',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))
    
    
    tracelist.append(go.Scatter(x=[pecaso_extrema.one_stage_best_batch.stage_one_factor/100*\
                                   max(pecaso_extrema.production_envelope.growth_rates)],
                                y=[pecaso_extrema.one_stage_best_batch.stage_two_factor/100*\
                                   max(pecaso_extrema.production_envelope.growth_rates)],
                                name='Best One Stage Process',
                                mode='markers', hoverinfo='text', showlegend=True,
                                marker=dict(color='#9DD7F5',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))
    
    tracelist.append(go.Scatter(x=[max(pecaso_extrema.production_envelope.growth_rates)],
                                y=[min(pecaso_extrema.production_envelope.growth_rates)],
                                name='Traditional Two Stage Process',
                                mode='markers', hoverinfo='text', showlegend=True,
                                marker=dict(color='#F16263',
                                            size=13,
                                            line=dict(width=0.5, color='black'))))

    fig = go.Figure(data=tracelist)
    
    fig.update_layout(hovermode='closest',
                      height=525,
                      width=465,
                      showlegend=True,
                      legend=dict(x=-0.2, y=-0.15, orientation='h',
                                  font=dict(size=14, color='black')))
    fig.update_xaxes(range=[0,1], 
                     dtick=0.2,
                     title_text='Stage 1 Growth Rate (1/h)',
                     title_font=dict(size=14, color='black'),
                     tickfont=dict(size=14, color='black'),
                     ticks='outside',
                     tickwidth=1.5,
                     title_standoff=0,
                     showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)
    
    fig.update_yaxes(range=[0,1], 
                     dtick=0.2,
                     title_text='Stage 1 Growth Rate (1/h)',
                     title_font=dict(size=14, color='black'),
                     tickfont=dict(size=14, color='black'),
                     ticks='outside',
                     tickwidth=1.5,
                     showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False)

    plot(fig)
    #if not os.path.exists('images'):
    #    os.mkdir('images')
    #pio.write_image(fig, 'images/heatmap_'+characteristic+'_objA_log_uptake.svg')   

# Fermentation Trajectory #

In [16]:
pecaso = pecaso_reduced_extrema
ts_suboptimal = pecaso.two_stage_suboptimal_batch
os_best = pecaso.one_stage_best_batch
ts_best = pecaso.two_stage_best_batch
ferm_list = [ts_suboptimal, os_best, ts_best]
titles = ['Traditional Two Stage<br>Process', 'Best One Stage<br>Process', 'Best Two Stage<br>Process']
fig = subplots.make_subplots(rows=1, cols=3, subplot_titles=titles, horizontal_spacing=0.1)
max_conc = max([(max([max(data) for data in ferm.data])) for ferm in ferm_list])
max_t = max([max(ferm.time) for ferm in ferm_list])
fig.update_annotations(dict(font=dict(size=18, color='black')))

for col, ferm in enumerate(ferm_list):
    if ferm:
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[0], name='Biomass Concentration<br>(g/L)',
                                    line={'color': '#CD7929', 'dash':'dash'}, legendgroup='Biomass',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col+1)
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[1], name='Substrate Concentration<br>(mmol/L)',
                                    line={'color': '#2278B5'}, legendgroup='Substrate',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col + 1)
        fig.append_trace(go.Scatter(x=ferm.time, y=ferm.data[2], name='Product Concentration<br>(mmol/L)',
                                    line={'color': '#5E9A42', 'dash':'dashdot'}, legendgroup='Product',
                                    showlegend=True if col == 2 else False, mode='lines'), 1, col + 1)
        if col != 1:
            fig.append_trace(go.Scatter(x=[ferm.optimal_switch_time]*30,
                                        y=np.linspace(0, 0.8*max_conc, 30),
                                        name='Optimal Switch Time', mode='lines',
                                        line={'color': 'black',
                                                'width': 2,
                                                'dash': 'dot'},
                                        showlegend=True if col == 2 else False), 1, col+1)
        fig['layout']['annotations'] = list(fig['layout']['annotations']) + \
                                       [go.layout.Annotation
                                        (dict(x=0.7*max_t,
                                              y=1.01*max_conc,
                                              xref='x'+str(col+1), yref='y'+str(col+1),
                                              text='Productivity=' + str(round(ferm.batch_productivity, 3)) +
                                              '<br>Yield=' + str(round(ferm.batch_yield, 3)) +
                                              '<br>End Titer=' + str(round(ferm.batch_titer,3)),
                                              showarrow=False, font=dict(size=12, color='black'),
                                              ax=-0, ay=-0))]



fig.update_layout(hovermode='closest',
                  height=475,
                  width=925,
                  showlegend=True,
                  legend=dict(x=-0.08, y=-0.15, orientation='h',
                              font=dict(size=14, color='black')))

fig.update_xaxes(range=[0,max_t], 
                 dtick=5,
                 title_font=dict(size=14, color='black'),
                 title_text='Time (h)',
                 tickfont=dict(size=14, color='black'),
                 ticks='outside',
                 tickwidth=1.5,
                 title_standoff=0,
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False,
                 showgrid=False)

fig.update_yaxes(range=[0,max_conc*1.08], 
                 title_text='Concentration<br>(mmol/L or g/L)',
                 title_font=dict(size=14, color='black'),
                 tickfont=dict(size=14, color='black'),
                 ticks='outside',
                 tickwidth=1.5,
                 showline=True, linewidth=1.5, linecolor='black', mirror=False, zeroline=False,
                 showgrid=False)


plot(fig)

#if not os.path.exists('images'):
#    os.mkdir('images')
#pio.write_image(fig, 'images/fermentation_characteristics_objA_log_uptake.svg')   